In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import matplotlib.pyplot as plt


import folium
from folium.plugins import HeatMap
import branca


from shapely.geometry import Point

In [2]:
url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json"  # Replace with the actual API endpoint
params = {
    'format': 'json',  # Output response format
    'api_key': '***',  # Your developer API key
    'status': 'all',  # Return stations that match the given status
    # 'access': 'public',  # Return stations with the given access type
    'fuel_type': 'ELEC', # Return stations that supply any of the given fuel types
    'state': 'CA',
    'country': 'US',
    
}

response = requests.get(url, params=params)
if response.status_code == 200:

    data = response.json()

In [3]:
df = pd.DataFrame(data['fuel_stations'])
sdf = df.copy()
sdf = sdf[sdf['city']=='San Diego']
sdf.head(1)

In [4]:
sdf[sdf['street_address'].str.contains('Judicial', case=False, na=False)]
# 616

In [5]:
import osmnx as ox
import geopandas as gpd

# Approximate coordinates for "Kearny Villa Rd & Century Park Ct"
intersection_location = (32.823413, -117.144259)

# Load the San Diego road network
G = ox.graph_from_place("San Diego, California, USA", network_type="drive")

# Extract the coordinates for the observation point
observation_lat = sdf.loc[616]["latitude"]
observation_lon = sdf.loc[616]["longitude"]
observation_location = (observation_lat, observation_lon)

# Find the nearest network nodes to the intersection and the observation point
intersection_node = ox.nearest_nodes(G, X=intersection_location[1], Y=intersection_location[0])
observation_node = ox.nearest_nodes(G, X=observation_location[1], Y=observation_location[0])

# Calculate the shortest path by road
route = ox.shortest_path(G, intersection_node, observation_node, weight="length")

# Calculate the total length of the route in meters
route_length_meters = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))

# Convert the length to miles (1 mile = 1609.34 meters)
route_length_miles = route_length_meters / 1609.34

# Print the distance
print(f"The distance of the route is {route_length_miles:.2f} miles.")

# Plot the map with the shortest path highlighted
fig, ax = ox.plot_graph_route(G, route, route_linewidth=4, node_size=0)